In [1]:
import nltk
import wikipedia

In [2]:
def named_entity_recognition(text):
    data = nltk.word_tokenize(text)
    data = nltk.pos_tag(data)
    data = nltk.ne_chunk(data)
    data = list(data)
    
    final = False
    
    for x in data:
        if type(x) is nltk.tree.Tree:
            final = x[0][0]
            
    return final

In [3]:
d = named_entity_recognition("Can you give me some info about Cahrles")

In [4]:
wikipedia.search("Bulgaria")

['Bulgaria',
 'Demographics of Bulgaria',
 'Sofia',
 'Bulgaria Air',
 "People's Republic of Bulgaria",
 'Balkan Wars',
 'Volga Bulgaria',
 'List of Bulgarian consorts',
 'Georgi Markov',
 'Provinces of Bulgaria']

In [5]:
bg = wikipedia.page("Bulgaria")

In [7]:
" ".join(bg.categories)

'All articles containing potentially dated statements All articles needing additional references All articles with dead external links All articles with unsourced statements All articles with vague or ambiguous time Articles containing Bulgarian-language text Articles containing potentially dated statements from 2012 Articles including recorded pronunciations (English) Articles needing additional references from March 2018 Articles with Bulgarian-language external links Articles with Curlie links Articles with dead external links from December 2017 Articles with hAudio microformats Articles with permanently dead external links Articles with unsourced statements from October 2011 Balkan countries Bulgaria CS1 Bulgarian-language sources (bg) CS1 French-language sources (fr) CS1 German-language sources (de) Coordinates on Wikidata Former empires Good articles Member states of NATO Member states of the Council of Europe Member states of the European Union Member states of the Union for the

In [37]:
len(bg.content[:1*10**3])

1000

In [38]:
len(bg.content)

52454

In [21]:
from elasticsearch import Elasticsearch
from datetime import datetime

es = Elasticsearch()
INDEX_NAME = "test"

if es.indices.exists(INDEX_NAME):
    print("deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))

request_body = {

  "settings": {
    "analysis": {
      "filter": {
        "english_stop": {
          "type":       "stop",
          "stopwords":  "_english_" 
        },
        "english_keywords": {
          "type":       "keyword_marker",
          "keywords":   ["example"] 
        },
        "english_stemmer": {
          "type":       "stemmer",
          "language":   "english"
        },
        "english_possessive_stemmer": {
          "type":       "stemmer",
          "language":   "possessive_english"
        }
      },
      "analyzer": {
        "english": {
          "tokenizer":  "standard",
          "filter": [
            "english_possessive_stemmer",
            "lowercase",
            "english_stop",
            "english_keywords",
            "english_stemmer"
          ]
        }
      }
    }
  }

}

print("creating '%s' index..." % (INDEX_NAME))
res = es.indices.create(index = INDEX_NAME, body = request_body)
print(" response: '%s'" % (res))

deleting 'test' index...
 response: '{'acknowledged': True}'
creating 'test' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test'}'


In [22]:
doc = {
    'author': 'victor',
    'text': 'Hello World',
    'timestamp': datetime.now(),
}

res = es.index(index=INDEX_NAME, doc_type='tweet', id=1, body=doc)
print(res['result'])

res = es.get(index=INDEX_NAME, doc_type='tweet', id=1)
print(res['_source'])

# es.indices.refresh(index=INDEX_NAME)


res = es.search(index=INDEX_NAME, body={"query": {"match_all": {}}})
# print(res)
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

created
{'author': 'victor', 'text': 'Hello World', 'timestamp': '2018-03-17T13:16:44.471407'}
Got 0 Hits:
